<img src="images/oasis-lmf-colour.png" alt="Oasis LMF logo" width="250" align="left"/>
<br><br><br>

# Excercise 3: Using code to adjust a model.

## Exercise goals
* Use Python code to adjust the model files for an example model

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
import pandas as pd
import jupyter_helper
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gpd
import math
import numpy as np
import json
import seaborn as sns
import folium
import os
from shapely.geometry import Point, Polygon
from numpy import linspace
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, FixedTicker, PrintfTickFormatter
from bokeh.plotting import figure
import branca.colormap as cm

# Output Bokeh charts to notebook, rather than opening a browser window
output_notebook()

## Read in the vulnerability data for the model

In [ ]:
vulnerabilities = pd.read_csv("./model_data/MEEQ/vulnerability_subset.csv")
vulnerabilities.head()

In [ ]:
vulnerabilities.vulnerability_id.unique()

In [ ]:
! rm -rf /tmp/analysis_test
! oasislmf model run -C oasislmf.json -r /tmp/analysis_test > /tmp/analysis.log
analysis_directory = "/tmp/analysis_test"
gul_oep1 = pd.read_csv(os.path.join(analysis_directory, "output", "gul_S1_leccalc_full_uncertainty_oep.csv"))
gul_oep1

Using the jupyter helper functions, we can edit the data directly in a worksheet then save it back to file.
Pick one vulnerability ID and intensity level and collapse all of the probability mass into one bin.  

In [ ]:
vulnerabilities_grid = jupyter_helper.show_df(vulnerabilities)
vulnerabilities_grid

In [ ]:
vulnerabilities = vulnerabilities_grid.get_changed_df()
vulnerabilities.to_csv("/tmp/vulnerability.csv", index=False)
! vulnerabilitytobin -d 52 < /tmp/vulnerability.csv > ./model_data/MEEQ/vulnerability.bin

In [ ]:
! vulnerabilitytobin -d 52 < ./model_data/MEEQ/vulnerability_subset.csv > ./model_data/MEEQ/vulnerability.bin

In [ ]:
! rm -rf /tmp/analysis_test
! oasislmf model run -C oasislmf.json -r /tmp/analysis_test > /tmp/analysis.log
gul_oep2 = pd.read_csv(os.path.join(analysis_directory, "output", "gul_S1_leccalc_full_uncertainty_oep.csv"))
gul_oep2

Adjusting the data by hand is tedious. 
The following code will adjust the vulnerability curves to put all the probability mass in the modal damage bin.

In [ ]:
# find the mode of the vulnerabilities
vulnerability_max = pd.DataFrame(
    {'probability' : vulnerabilities.groupby(
        ['vulnerability_id', 'intensity_bin_id'] )['probability'].max()
    }).reset_index()
adjusted_vulnerabilities = vulnerabilities.merge(
    vulnerability_max, 
    left_on=['vulnerability_id', 'intensity_bin_id', 'probability'], 
    right_on=['vulnerability_id', 'intensity_bin_id', 'probability'])
adjusted_vulnerabilities.probability = 1.0
adjusted_vulnerabilities

In [ ]:
adjusted_vulnerabilities.to_csv("/tmp/vulnerability.csv", index=False)
! vulnerabilitytobin -d 52 < /tmp/vulnerability.csv > ./model_data/MEEQ/vulnerability.bin

In [ ]:
! rm -rf /tmp/analysis_test
! oasislmf model run -C oasislmf.json -r /tmp/analysis_test > /tmp/analysis.log
gul_oep3 = pd.read_csv(os.path.join(analysis_directory, "output", "gul_S1_leccalc_full_uncertainty_oep.csv"))
gul_oep3